In [1]:

## tts engine 함수 호출
## text to speech
import pyttsx3

def text_to_speech(language):
    """
    text_to_speech engine 구동 함수 \\
    language = English, Korean
    """
    engine = pyttsx3.init()
    try:
        voices = engine.getProperty("voices")
        for v in voices:
            if language in v.name:
                target_tts_index = voices.index(v)
                engine.setProperty("voice", voices[target_tts_index].id)
                print(f"TTS VOICE IS SET TO {v.name}.")
                break

    except:
        print("not use tts.")
    
    return engine

language = "English"
engine = text_to_speech(language=language)

TTS VOICE IS SET TO Microsoft Zira Desktop - English (United States).


In [2]:

## google drive, google sheet api auth

#? pip install auth2client
#? pip install gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread  ##  connection to google sheets

json_key_path = "./test1203_google_drive.json"

def auth_google_drive_sheet(json_key_path):
    scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
    ]

    credential = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
    gc = gspread.authorize(credential)
    
    return gc

gc = auth_google_drive_sheet(json_key_path)

In [3]:

## 구글 Drive > sheet 에서 공부 소스 가져오기
import pandas as pd

spreadsheet_url = "https://docs.google.com/spreadsheets/d/17vjhn12bMNCj4cOHMDXNzQEOD430JsA0LKbtGm71MHQ/edit#gid=0"

def get_study_source_and_history(gc, spreadsheet_url):
    
    ## 구글 drive 영어공부용 google sheet 파일 URL로 열기
    doc = gc.open_by_url(spreadsheet_url)

    wordsheet = doc.worksheet("WordDB")  ## 공부용 시트
    study_history_sheet = doc.worksheet("TestHistoryDB")  ## 공부 히스토리

    ## 공부 DB DataFrame
    words_list_of_dicts = wordsheet.get_all_records()
    words_df = pd.DataFrame(words_list_of_dicts)

    ## 공부 이력 DB DataFrame
    record_list_of_dicts = study_history_sheet.get_all_records()
    study_history_df = pd.DataFrame(record_list_of_dicts)
    
    return words_df, study_history_df, study_history_sheet

In [4]:

## 공부 타입별 공부 범위 가져오기
import random
from datetime import datetime
import json

def get_study_range_data_by_study_type(words_df, study_history_df, study_range, is_review):

    test_word_index_list = []

    ## 원하는 공부 범위
    target_words_df = words_df[words_df["Day"] == f"day{study_range}"].reset_index(drop=True)

    ## 리뷰 원하는 경우 
    if is_review == 1:

        ## 공부 원하는 범위에 공부한 기록 DB 가져오기
        target_study_history_df = study_history_df[study_history_df["day"] == study_range]
        study_history_data_length = len(target_study_history_df)

        if study_history_data_length == 0:
            print("You have no history of studying this range.")
            print("It starts for all content within the range you select.")

            if len(target_words_df) == 0:
                print("There are no words to study.")
                pass
            
            else:
                test_word_index_list = list(target_words_df.index)
                random.shuffle(test_word_index_list)

            return target_words_df, test_word_index_list
        
        elif study_history_data_length > 0:
            test_word_index_list = json.loads(target_study_history_df["wrong_words_list"].iloc[-1])

            return target_words_df, test_word_index_list
            
    elif is_review == 0:
        if len(target_words_df) == 0:
            print("There are no words to study.")
            pass
        
        else:
            test_word_index_list = list(target_words_df.index)
            random.shuffle(test_word_index_list)
        
        return target_words_df, test_word_index_list
    
    else:
        print("Check the is_review variable value.")
        return target_words_df, test_word_index_list

In [5]:

def work_word_study(engine, target_words_df, test_word_index_list, is_review, study_history_df, study_history_sheet, study_range, answer=None):
    
    ## 공부 대상 단어 갯수
    total_study_word_count = len(test_word_index_list)

    r_dict = {}
    right_words_list = []
    wrong_words_list = []

    if total_study_word_count == 0:
        print("It has no study words.")
    else:
        ## 공부 결과
        for i in test_word_index_list:
            t_df = target_words_df.iloc[i]
            word = t_df["Word"]
            solution = t_df["Meaning"]

            ## question
            print(f"fWhat does this word mean? : {word}")
            engine.say(word)
            engine.runAndWait()

            ## answer
            answer = input("Enter the word meaning.")

            if answer == "exit":
                print("Stop the test.")
                break
            
            elif answer != '':
                if answer in solution:
                    print(f"solution : {solution}")
                    print(f"answer : {answer}")
                    print("result : 정답")
                    if i not in right_words_list:
                        right_words_list.append(i)

                elif answer not in solution:
                    print(f"solution : {solution}")
                    print(f"answer : {answer}")
                    print("result : Not 정답")
                    if i not in wrong_words_list:
                        wrong_words_list.append(i)
            elif answer == '':
                print(f"solution : {solution}")
                print("Answer is not None.")
                if i not in wrong_words_list:
                    wrong_words_list.append(i)
                    print("reuslt : Not 정답, Empty Answer")
                pass
    
    ## 공부 완료한 시간
    test_date = datetime.now()
    r_dict = {
        "test_date" : test_date.strftime("%m/%d/%Y, %H:%M:%S"),
        "day" : study_range,
        "total_word_count" : total_study_word_count,
        "test_word_index_list" : f"{test_word_index_list}",
        "right_words_list" : f"{right_words_list}",
        "wrong_words_list" : f"{wrong_words_list}",
        "score" : len(right_words_list),
        "is_review" : is_review,
    }

    return r_dict, answer

## 학습 결과 google sheet 저장
def record_study_result(study_history_df, r_dict, study_history_sheet):
    r_list = list(r_dict.values())

    length_study_history_df = len(study_history_df)
    insert_row_num = length_study_history_df + 1
    study_history_sheet.append_rows(values=[r_list], table_range=f"A{insert_row_num}:H{insert_row_num}", value_input_option="RAW")

In [10]:

## User Setting

## 공부할 범위
study_range = 16  ## 1일차부터 30일까지

## 공부 종류
is_review = 0  ## review 여부 틀린 단어만 공부 = 1, 모든 단어 공부 = 0
words_df, study_history_df, study_history_sheet = get_study_source_and_history(gc, spreadsheet_url)
target_words_df, test_word_index_list = get_study_range_data_by_study_type(words_df, study_history_df, study_range=study_range, is_review=is_review)
r_dict, answer = work_word_study(engine, target_words_df, test_word_index_list, is_review, study_history_df, study_history_sheet, study_range)
if answer == "exit":
    pass
elif len(test_word_index_list) == 0:
    pass
else:
    record_study_result(study_history_df, r_dict, study_history_sheet)


fWhat does this word mean? : contact
solution : ~와 연락하다,
 (상업적 목적으로)교섭하고 있는 사람
answer : 접촉하다
result : Not 정답
fWhat does this word mean? : bargain
solution : 특가품, 흥정
Answer is not None.
reuslt : Not 정답, Empty Answer
fWhat does this word mean? : discount
solution : 할인
answer : 할인
result : 정답
fWhat does this word mean? : confirmation
Stop the test.
